# Imports:

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import itertools
import numpy.matlib
import time

# Data Generation:

We generate samples X that are uniformly distributed on [-1,1]^8 and then compupte the output Y of our regression model.

In [3]:
n = 100000 # Number of samples
m = 1000 # Number of samples for validation/testing dataset
d = 8 # Dimension of feature variable vector
sigma = 0.01

np.random.seed(1) # Set seed for reproducibility
X = np.random.uniform(-1,1,(n + 2*m,d)) # 8 dimensional random vector X uniformly distributed on [-1,1]
eps = np.random.normal(0,sigma,n + 2*m) # Gaussian noise

Y = 8 + X[:,0]**2 + X[:,1]*X[:,2] + np.cos(X[:,3]) + np.exp(X[:,4]*X[:,5]) + 0.1*X[:,6] + eps # Output computation
Y = Y.reshape((-1,1)) # Transpose output vector

data = np.concatenate((X,Y), axis=1) # First 8 columns as input and last column as output
data_df = pd.DataFrame(data) # Create dataframe of numpy matrix
data_df.columns = ['X_1','X_2','X_3','X_4','X_5','X_6','X_7','X_8','Y']
data_df

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,Y
0,-0.165956,0.440649,-0.999771,-0.395335,-0.706488,-0.815323,-0.627480,-0.308879,10.218395
1,-0.206465,0.077633,-0.161611,0.370439,-0.591096,0.756235,-0.945225,0.340935,9.505227
2,-0.165390,0.117380,-0.719226,-0.603797,0.601489,0.936523,-0.373152,0.384645,10.492027
3,0.752778,0.789213,-0.829912,-0.921890,-0.660339,0.756285,-0.803306,-0.157785,9.026440
4,0.915779,0.066331,0.383754,-0.368969,0.373002,0.669251,-0.963423,0.500289,10.966825
...,...,...,...,...,...,...,...,...,...
101995,-0.965163,-0.868066,0.687381,0.737635,0.201860,0.130245,-0.953401,0.992092,10.001159
101996,0.801107,0.875722,-0.990787,0.911076,0.353634,-0.470982,-0.985444,-0.509583,9.140565
101997,0.126903,-0.399359,-0.606135,-0.742466,-0.001000,-0.924128,-0.592043,0.104867,9.935730
101998,-0.102098,-0.232576,-0.926793,-0.189997,-0.359941,-0.060435,-0.958450,0.175319,10.154265


Now we split the data set into training, validation and testing samples:

In [4]:
# Training data

x_train = pd.DataFrame(data[:n,0:d]).to_numpy()
y_train = pd.DataFrame(data[:n,d]).to_numpy()

# Validation data

x_val = pd.DataFrame(data[n:n + m,0:d]).to_numpy()
y_val = pd.DataFrame(data[n:n + m,d]).to_numpy()

# Testing data

x_test = pd.DataFrame(data[n + m:n + 2*m,0:d]).to_numpy()
y_test = pd.DataFrame(data[n + m:n + 2*m,d]).to_numpy()

# Neural Network Model

We construct our neural network as a fully-connected, single-layer, feed-forward, regression neural network and compile it with the default Adam optimizer and the mean squared error loss function.

In [5]:
tf.random.set_seed(1) # Set seed for reproducibility

# Neural network model

neurons = 25 # 25 neurons in the hidden layer

NN_model = tf.keras.models.Sequential()
NN_model.add(tf.keras.layers.Dense(neurons, activation=tf.nn.sigmoid, input_dim = d)) 
NN_model.add(tf.keras.layers.Dense(1))

NN_model.compile(optimizer='Adam', loss='mse')

# Training process

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=10**(-5), patience=5) # Early stopping criterion

NN_model.fit(x_train, y_train, batch_size=32, epochs=150, validation_data=(x_val, y_val), callbacks=[callback], verbose=0) # Training process

Epoch 1/150
3125/3125 [==============================] - 2s 644us/step - loss: 7.6113 - val_loss: 0.3413
Epoch 2/150
3125/3125 [==============================] - 2s 520us/step - loss: 0.3051 - val_loss: 0.2597
Epoch 3/150
3125/3125 [==============================] - 2s 535us/step - loss: 0.2271 - val_loss: 0.1951
Epoch 4/150
3125/3125 [==============================] - 2s 533us/step - loss: 0.1878 - val_loss: 0.1573
Epoch 5/150
3125/3125 [==============================] - 2s 513us/step - loss: 0.1115 - val_loss: 0.0622
Epoch 6/150
3125/3125 [==============================] - 2s 516us/step - loss: 0.0489 - val_loss: 0.0391
Epoch 7/150
3125/3125 [==============================] - 2s 513us/step - loss: 0.0284 - val_loss: 0.0191
Epoch 8/150
3125/3125 [==============================] - 2s 509us/step - loss: 0.0174 - val_loss: 0.0139
Epoch 9/150
3125/3125 [==============================] - 2s 514us/step - loss: 0.0131 - val_loss: 0.0120
Epoch 10/150
3125/3125 [==============================]

Now we evaluate our model by computing the loss on the validation and testing data set.

In [6]:
val_loss = NN_model.evaluate(x_val, y_val)
print("Loss on validation set is",val_loss)

test_loss = NN_model.evaluate(x_test, y_test)
print("Loss on test set is",test_loss)

32/32 [==============================] - 0s 734us/step - loss: 2.1917e-04
Loss on validation set is 0.00021917418052908033
32/32 [==============================] - 0s 542us/step - loss: 2.2498e-04
Loss on test set is 0.00022498385806102306


# Linear Regression Model

We compare the approximation quality of our neural network to an alternative linear regression model:

In [7]:
LR_model = LinearRegression() # Linear regression model provided from scikit-learn
LR_model.fit(x_train,y_train) # Fit training data set

# Print model parameters

param = np.append(LR_model.coef_,LR_model.intercept_)
param_df = pd.DataFrame(param.transpose(), ['X_1','X_2','X_3','X_4','X_5','X_6','X_7','X_8','intercept'], columns=['Coefficient'])
print('Linear regression model parameters: \n',param_df)

Linear regression model parameters: 
            Coefficient
X_1          -0.005766
X_2          -0.000137
X_3           0.005428
X_4          -0.004201
X_5           0.000292
X_6          -0.006256
X_7           0.101468
X_8          -0.006968
intercept    10.230597


Fitting the model and computing the MSE on the test data set:

In [8]:
y_pred = LR_model.predict(x_test) # Compute predictions for test set
print('Test MSE:', metrics.mean_squared_error(y_test,y_pred))

Test MSE: 0.3383625773788169


# Empirical Neural Network Test Statistic

Now we compute the empirical neural network test statistic. For that we first calculate the partial derivatives with respect to x_j over the training data set. First we compute the partial derivatives of the neural network with respect to each input variable:

In [9]:
x = tf.constant(x_train, dtype=tf.float32) # We regard the gradient over the training data set

with tf.GradientTape(persistent=True) as tape: # Calculate gradient of the neural network
    tape.watch(x)
    y = NN_model(x)

gradient = tape.gradient(y, x).numpy()

del tape # Drop the reference to the tape

And now we compute the corresponding empirical expectations which represents the empirical neural network test statistic:

In [10]:
NN_test_stat = 1/n * (gradient**2).sum(axis=0) # Estimator of functional of neural network

np.set_printoptions(precision=6, suppress=True) # Set precision of output

print('Empirical neural network test statistic with respect to each variable: \n',NN_test_stat)

Empirical neural network test statistic with respect to each variable: 
 [1.290392 0.331923 0.33015  0.266212 0.492402 0.491897 0.010313 0.000017]


# Leave-One-Out Metric

We compare the performance of our test statistic to the leave-one-out metric, which also evaluates variable influence. This metric is constructed by calculating the difference between the loss of the original model and the loss of the same model but fitted without the variable of interest.

In [11]:
leave_one_out_metric = np.zeros(8) # Initialize leave one out metric list

for j in range(8): # Fit 8 neural networks, each without one different input variable
    print('Variable-index j =',j)
    # Heating issues of my hardware, added cool-down phase
    # Drop values of the j.th variable X_j

    x_train_j = np.delete(x_train,j,1)
    x_val_j = np.delete(x_val,j,1)
    x_test_j = np.delete(x_test,j,1)

    # Neural network model

    NN_model_j = tf.keras.models.Sequential()
    NN_model_j.add(tf.keras.layers.Dense(25, activation=tf.nn.sigmoid, input_dim = d - 1))
    NN_model_j.add(tf.keras.layers.Dense(1))

    NN_model_j.compile(optimizer='Adam', loss='mse')

    # Training process

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=10**(-5), patience=5) # Early stopping criterion

    NN_model_j.fit(x_train_j, y_train, batch_size=32, epochs=150, validation_data=(x_val_j, y_val), callbacks=[callback], verbose=0)

    # Compute test MSE and difference to standard NN_model

    test_loss_j = NN_model_j.evaluate(x_test_j, y_test)
    leave_one_out_metric[j] = abs(test_loss - test_loss_j)

Variable-index j = 0
32/32 [==============================] - 0s 512us/step - loss: 0.0928
Variable-index j = 1
32/32 [==============================] - 0s 636us/step - loss: 0.1228
Variable-index j = 2
32/32 [==============================] - 0s 546us/step - loss: 0.1257
Variable-index j = 3
32/32 [==============================] - 0s 596us/step - loss: 0.0201
Variable-index j = 4
32/32 [==============================] - 0s 657us/step - loss: 0.1456
Variable-index j = 5
32/32 [==============================] - 0s 962us/step - loss: 0.1488
Variable-index j = 6
32/32 [==============================] - 0s 686us/step - loss: 0.0037
Variable-index j = 7
32/32 [==============================] - 0s 668us/step - loss: 2.4394e-04


In [12]:
print('Leave one out metric with respect to each variable: \n',leave_one_out_metric)

Leave one out metric with respect to each variable: 
 [0.092588 0.12255  0.125523 0.019852 0.145383 0.148606 0.00344  0.000019]


# Quantile Estimation

We now estimates the quantile of the limiting distribution of the neural network test statistic. The first approach is using the series representation and the second approach uses the discretization method.

# Series Representation Approach

We frist compute the truncated sums in the nominator and denominator. The truncation order N indicates that the sum over all elements z = (z_1,...,z_d) in Z^d is truncated in a way such that we omitted z with |z_j| > N for some 1 ≤ j ≤ d.

Setting computation constants for our example:

In [13]:
N = 1 # Truncation order
m_N = 10000 # Number of samples
s = d//2 + 2 # Sobolev norm index

Genrate samples of the chi-squared distribution:

In [14]:
start = time.time() # Measuring the computing time of the chi-squared sampling process

x_iz = np.random.chisquare(1,(m_N,2**d,(2*N + 1)**d))

end = time.time()
print('Sampling process took',end - start,'seconds.')

Sampling process took 12.811657905578613 seconds.


Generate and label elements in the sets {0,1}^d and Z^d such that we can iterate through them using the natural numbers:

In [15]:
z_nums = np.zeros((((2*N + 1)**d),d)) # Placeholder for elements in Z^d
itr = 0 # Iteration counter
for z in itertools.product(range(-N,N + 1), repeat=d): # This line generates the actual tuples
    z_nums[itr,:] = np.array(z) # Placing the generated tuple
    itr = itr + 1

i_nums = np.zeros(((2**d),d)) # Placeholder for elements in {0,1}^d
itr = 0 # Resetting iteration counter
for i in itertools.product(range(2), repeat=d): # This line generates the actual tuples
    i_nums[itr,:] = np.array(i) # Placing the generated tuple
    itr = itr + 1

print('Finished generating indexes. Shapes:',z_nums.shape,i_nums.shape)

Finished generating indexes. Shapes: (6561, 8) (256, 8)


Computing the weigths d_{i,z}^2:

In [16]:
# First compute weights d_{i,z}^2 for a fixed i \in {0,1}^d

d_z = np.zeros((2*N + 1)**d) # Placeholder for weights d_{i,z}^2 for fixed i

for a in itertools.product(range(s), repeat=d): # Generate multiindex alpha
    if sum(list(a)) <= s: # Only consider |alpha| <= s
        d_z = d_z + np.prod((np.pi*z_nums)**(np.multiply(2,a)), axis=1) # Increase sum

# Compute d_{i,z}^2

d_iz = np.matlib.repmat(d_z,256,1) # The weights are identical for different i so we replicate them

print('Shape of the weight array:',d_iz.shape)

Shape of the weight array: (256, 6561)


Now we can compute the truncated sum in the denominator:

In [17]:
denom_sum = np.sum(x_iz/d_iz, axis=(1,2))
print('Different samples of the truncated sum in the denominator: \n',denom_sum.shape)

Different samples of the truncated sum in the denominator: 
 (100,)


We continue with calculating the weights alpha_{i,z,j}^2

In [18]:
alpha_zj = (np.pi*z_nums)**2 # alpha_{z,j}^2 for fixed i and every j

print(alpha_zj.shape)

alpha_izj = np.tile(alpha_zj,(2**d,1,1)) # Identical for different i, so we copy it 2^d times

print(alpha_izj.shape)

(6561, 8)
(256, 6561, 8)


Now we compute the truncated sum in the nominator

In [19]:
frac = x_iz/(d_iz**2)
prod = np.einsum('abc,xab->xabc', alpha_izj,frac)

start = time.time() # Measuring the computing time of the nominator sum

nom_sum = np.sum(prod, axis=(1,2))

end = time.time()

print('Computing nominator sum took',end - start,'seconds.')
print('Different samples of the truncated sum in the nominator with respect to each variable: \n',nom_sum.shape)

Computing nominator sum took 33.301169872283936 seconds.
Different samples of the truncated sum in the nominator with respect to each variable: 
 (100, 8)


With that we can already calculate samples from the limiting distribution and scaling factor B = 1. Then we just have to scale the samples with B^2.

In [20]:
unscaled_Z = nom_sum/denom_sum[:,np.newaxis] # Samples from the limiting distribution with B = 1
print('Unscaled samples of the limiting distribution with respect to each variable: (very small) \n',unscaled_Z.shape)

Unscaled samples of the limiting distribution with respect to each variable: (very small) 
 (100, 8)


# Calculating B

To obtain samples from the series representation it remains to choose the scaling factor B^2. Therefore we need a constant B that dominates the Sobolev norm of the unknown regression function f_0 while being as small as possible. Thus one could set B equal to the Sobolev norm for f_0. Typically f_0 is unknown but we start with explicitly calculating its Sobolev norm partly by hand and partly by numerical integration. We will use this as a reference for our B estimator.

To compute the Sobolev norm, we have to compute the squared L^2 norm for every partial derivative explicitly. 

In [21]:
ev = np.zeros(28) # Placeholder for the squared L^2 norms of the partial derivatives, only 28 are not zero

We start with the squared L^2 norm of f_0. We estimate it using the empirial expectation of f_n on the training dataset.

In [22]:
f_n = NN_model.predict(x_train)[:,0]
ev[0] = sum(f_n**2)
print('Estimation for the squared L^2 norm of f_0:',ev[0])

Estimation for the squared L^2 norm of f_0: 10501646.59098053


The next squared partial derivative expectations were calculated by hand. By the symmetry of certain variables (e.g. X_5 and X_6) we double or tripple certain expectations that occur two or three times in the sum.

In [23]:
ev[1] = 4/3
ev[2] = 4
ev[3] = 2 * 1/3 # 2 times
ev[4] = 1
ev[5] = 3 * (0.5 - 0.25*np.sin(2)) # 3 times
ev[6] = 3 * (np.cos(1)*np.sin(1)/2 + 0.5) # 3 times
ev[7] = 0.01 

We compute the missing partial derivatives using scipy's integration tools. If some expectations occur more than once in the sum, we scale the integrand by the number of occurances.

In [24]:
from scipy.integrate import quad
from scipy.integrate import dblquad
from scipy.integrate import tplquad

In [25]:
# y = x_5, x = x_6
# Probability density function: 1/(2^d), so only 1/4 for the two variables X_5 and X_6
# Define the integrands:

def int1(y, x):
    return ((y*np.exp(x*y))**2)/4

def int2(y, x):
    return (((y**2)*np.exp(x*y))**2)/4

def int3(y, x):
    return (((y**3)*np.exp(x*y))**2)/4

def int4(y, x):
    return (((y**4)*np.exp(x*y))**2)/4

def int5(y, x):
    return (((y**5)*np.exp(x*y))**2)/4

def int6(y, x):
    return (((y**6)*np.exp(x*y))**2)/4

def int7(y, x):
    return (((1 + x*y)*np.exp(x*y))**2)/4

def int8(y, x): # 2 times
    return (((2*y + (y**2)*x)*np.exp(y*x))**2)/2

def int9(y, x): # 2 times
    return (((2*(y**2) + x*(y**3))*np.exp(y*x))**2)/2

def int10(y, x): # 2 times
    return (((2*(y**3) + x*(y**4))*np.exp(y*x))**2)/2

def int11(y, x): # 2 times
    return (((2*(y**4) + x*(y**5))*np.exp(y*x))**2)/2

def int12(y, x): 
    return (((2 + 4*y*x + (y*x)**2)*np.exp(y*x))**2)/4

def int13(y, x): # 2 times
    return (((6*y + 6*(y**2)*x + (y*x)**2)*np.exp(y*x))**2)/2

def int14(y, x): # 2 times
    return (((12*(y**2) + 8*(y**3)*x + (y**4)*(x**2))*np.exp(y*x))**2)/2

def int15(y, x):
    return (((6 + 18*y*x + 9*((y*x)**2) + (y*x)**3)*np.exp(y*x))**2)/4

# List of the integrands for iteration:
int_list = [int1, int2, int3, int4, int5, int6, int7, int8, int9, int10, int11, int12, int13, int14, int15]

# Computing the integrals:
for i in range(15):
    ev[13 + i] = dblquad(int_list[i], -1, 1, lambda x: -1, lambda x: 1)[0]

Now we just have to sum all expectations together and take the square root to obtain an estimator for the Sobolev norm of f_0

In [26]:
sob_norm = np.sqrt(sum(ev)) # Desired B value, Sobolev norm of f_0
print('Estimation for the sobolev norm of f_0:',sob_norm)

Estimation for the sobolev norm of f_0: 3240.7039410538114


# B Estimator

In [27]:
t = 100
zx_comb = np.einsum('ab,xb->axb', z_nums, x_train[:t,:]) # Every possible combination of z and x_train values
# Compute cosine and sine values of each combination
cos_val = np.cos(np.pi*zx_comb)
sin_val = np.sin(np.pi*zx_comb)

not_i_nums = 1 - i_nums

izx_cos_comb = np.einsum('ab,xyb->axyb', not_i_nums, cos_val)
izx_sin_comb = np.einsum('ab,xyb->axyb', i_nums, cos_val)

phi_array = izx_cos_comb + izx_sin_comb

phi = np.prod(phi_array, axis=3)

phi.shape

model_val = NN_model.predict(x_train[:t,:])[:,0]
model_val.shape

skalarprod = np.sum(phi*model_val, axis=2)/t

B = np.sqrt(np.sum(d_iz*(skalarprod**2)))
print(B)

"\nt = 100\nzx_comb = np.einsum('ab,xb->axb', z_nums, x_train[:t,:]) # Every possible combination of z and x_train values\n# Compute cosine and sine values of each combination\ncos_val = np.cos(np.pi*zx_comb)\nsin_val = np.sin(np.pi*zx_comb)\n\nnot_i_nums = 1 - i_nums\n\nizx_cos_comb = np.einsum('ab,xyb->axyb', not_i_nums, cos_val)\nizx_sin_comb = np.einsum('ab,xyb->axyb', i_nums, cos_val)\n\nphi_array = izx_cos_comb + izx_sin_comb\n\nphi = np.prod(phi_array, axis=3)\n\nphi.shape\n\nmodel_val = NN_model.predict(x_train[:t,:])[:,0]\nmodel_val.shape\n\nskalarprod = np.sum(phi*model_val, axis=2)/t\n\nB = np.sqrt(np.sum(d_iz*(skalarprod**2)))\nprint(B\n"

# Calculating B by Sampling Neural Networks

This method is computationally inefficient as it requires fitting several neural networks to different data sets. Later we require fitting multiple networks again for estimating the test's performance and since these tasks coincide, we refer to the section "Performance of the test" for the computation of the neural network test statistic samples. Using these samples we can calculate the empirical expectation and determine the B estimator:

In [28]:
B = 643.3893036066657 # Obtained from later implementation

# Multiply unscaled samples from the limiting distribution by B^2

Z = (B**2)*unscaled_Z
print('Samples of the limiting distribution of the neural network test statistic: \n',Z.shape)

Samples of the limiting distribution of the neural network test statistic: 
 (100, 8)


Compute the quantiles:

In [29]:
order_Z = np.sort(Z, axis=0) # Order statistics of samples

alpha = 0.05 # Confidence Level

# Find empirical quantile

intervals = np.linspace(0,1,m_N + 1) # Array m_N probability values from 0 increasing to 1
i = np.searchsorted(intervals, 1 - alpha) # Find index where 1 - alpha would be sorted in to maintain increasing order
print('Order statistic index i =',i)
SNNquantiles = order_Z[i - 1][:] # Quantiles by using sampled neural networks to calculate B
print('Quantiles Z_i with respect to each variable: \n',SNNquantiles)

Order statistic index i = 95
Quantiles Z_i with respect to each variable: 
 [0.000898 0.000889 0.000881 0.000902 0.000893 0.000932 0.000909 0.000898]


# Test Results (Series Representation)

In [30]:
test_bool_array = ((neurons**2)*NN_test_stat > SNNquantiles).astype(int) # Check if the scaled neural network test statistic  exceeds the quantile
print('Test result:',test_bool_array)

Test result: [1 1 1 1 1 1 1 1]


# Discretization Method

This method is computationally far superior as it does not require re-fitting the neural network. To obtain samples from the limiting distribution, we need to identify the argmax of the Gaussian process indexed by the function space \Theta. We achieve this by approximation the infinite dimensional function space with an epsilon cover. This cover only requires a finite number of functions from \Theta. We randomly generate functions from \Theta and by generating a large enough number, the probabilty of covering \Theta increases. For our implementation we use 500 randomly generated functions, which in our case are neural networks with randomly generated parameters. Now comes the sampling process:

In [31]:
nrf = 500 # Number of random functions we sample
rnd_fcts = [] # Placeholder for random functions

for i in range(nrf): # Generate neural networks with randomly sampled parameters
    rnd_fcts.append(tf.keras.models.Sequential())
    rnd_fcts[i].add(tf.keras.layers.Dense(25, activation=tf.nn.sigmoid, input_dim = d, kernel_initializer='glorot_normal', bias_initializer='glorot_normal')) # Glorot normal distribution for model parameters
    rnd_fcts[i].add(tf.keras.layers.Dense(1))

The next step is to sample from the limiting Gaussian process indexed by our 500 functions. The mean is determined by zero so it remains to compute the covariance matrix. We start by evaluating the 500 functions h on the training data set:

In [32]:
h_evals = np.zeros((nrf,n)) # Placeholder for evaluation of random functions h

for i in range(nrf): # Evaluate random functions
    h_evals[i] = rnd_fcts[i].predict(x_train)[:,0]

With that we can determine the covariance estimator:

In [33]:
h_sum = h_evals @ h_evals.T # Compute sum of h_i(X_k)h_j(X_k) for fixed i,j
cov = 4*(sigma**2)*h_sum/n # Calculate covariance matrix

Now we sample 1000 times from the 500 dimensional multivariate Gaussian distribution and identify the argmax for each of the 1000 sample vectors:

In [34]:
zeros = np.zeros(nrf) # Mean vector of the Gaussian process
nas = 10000 # Number of argmax samples

g_process = np.random.multivariate_normal(zeros, cov, nas) # Sample from limiting Gaussian process
arg_idx = np.argmax(g_process, axis=1) # Find argmax index

The next step is to compute the expected value of the squared partial derivatives of the argmax function. We first evaluate the gradient on the training data set for each of our 500 functions:

In [35]:
gradients = np.zeros((nrf,n,d)) # Placeholder for gradients of all 500 functions
x = tf.constant(x_train, dtype=tf.float32) # We regard the gradient over the training data set

for i in range(nrf): # Calculate gradients of all 500 functions
    with tf.GradientTape(persistent=True) as tape: # Calculate gradient of the neural network
        tape.watch(x)
        y = rnd_fcts[i](x)

    gradients[i] = tape.gradient(y, x).numpy()

del tape  # Drop the reference to the tape

Now we only consider the evaluated gradients for functions that actually appear as an argmax:

In [36]:
argm_gradients = np.zeros((nas,n,d)) # Placeholder for gradients of argmax functions only

for i in range(nas): # Insert gradients of argmax functions
    argm_gradients[i] = gradients[arg_idx[i]]

We calculate the estimator for the expection of the squared partial derivatives:

In [37]:
Z = 1/n * (argm_gradients**2).sum(axis=1)
print(Z.shape)

(1000, 8)


We now have 1000 samples of the limiting distribution for each coordinate 1 ≤ j ≤ d. Therefore we now determine the empirical quantiles:

In [38]:
intervals = np.linspace(0,1,m_N + 1) # Array m_N probability values from 0 increasing to 1
i = np.searchsorted(intervals, 1 - alpha) # Find index where 1 - alpha would be sorted in to maintain increasing order
print('Order statistic index i =',i)
quantiles = order_Z[i - 1][:]
print('Quantiles Z_i with respect to each variable: \n',quantiles)

Order statistic index i = 95
Quantiles Z_i with respect to each variable: 
 [0.000898 0.000889 0.000881 0.000902 0.000893 0.000932 0.000909 0.000898]


# Test Results (Discretization Method):

In [39]:
test_bool_array = (25*NN_test_stat > quantiles).astype(int)
print('Test result:',test_bool_array)

Test result: [1 1 1 1 1 1 1 0]


# Performance of the Test

To evaluate the performance of the test, we estimate its power and size by performing it on 100 alternative training data sets. Recall:

**Power**: Probability that the null hypothesis is rejected when the alternative is true.\
**Size**: Probabilty that the null hypothesis is rejected when the null is true.

Therefore the expected value of the test statistic for X_8 represents its power and the expected value of the test for all other feature variables represents its size. We first generate the new 100 alternative data sets:

In [40]:
num = 250 # Number of data sets
# Other constants are defined in the first data generation section

X = np.random.uniform(-1,1,(num*n,d)) # 100*n samples of 8 dimensional random vector X uniformly distributed on [-1,1]
eps = np.random.normal(0,sigma,num*n) # Gaussian noise

Y = 8 + X[:,0]**2 + X[:,1]*X[:,2] + np.cos(X[:,3]) + np.exp(X[:,4]*X[:,5]) + 0.1*X[:,6] + eps # Output computation
Y = Y.reshape((-1,1)) # Transpose output vector

data = np.concatenate((X,Y), axis=1) # First 8 columns as input and last column as output
data_df = pd.DataFrame(data) # Create dataframe of numpy matrix
data_df.columns = ['X_1','X_2','X_3','X_4','X_5','X_6','X_7','X_8','Y']
data_df

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,Y
0,-0.044155,0.089165,-0.555221,0.712626,-0.252611,0.505551,-0.704385,0.075142,9.511502
1,-0.076891,0.511528,-0.900746,0.826395,0.213613,0.146388,0.324458,-0.106687,9.283602
2,0.720737,0.925285,0.910770,0.057433,-0.024372,0.580635,-0.378254,-0.950956,11.312323
3,0.760672,0.057912,-0.480058,0.595180,0.667535,0.747551,-0.075823,-0.474159,11.025729
4,0.403479,0.204379,0.367499,0.264937,0.873847,-0.493808,-0.055887,0.891440,9.840353
...,...,...,...,...,...,...,...,...,...
999995,-0.083593,-0.950088,-0.336528,-0.872823,-0.772132,-0.867780,-0.610100,-0.327872,10.861105
999996,-0.677511,0.059008,-0.699660,0.012592,-0.356464,0.871729,-0.916914,0.751804,10.061728
999997,0.266204,-0.657625,-0.860812,0.114747,0.218639,-0.260168,-0.788328,0.656879,10.457416
999998,-0.362661,-0.309295,0.846551,0.101806,-0.449618,0.621645,0.452097,0.549557,9.674069


Split generated data into 100 training data set

In [41]:
x_train = np.array(np.split(pd.DataFrame(data[:,0:8]).to_numpy(), num)) # x_train as array of 100 training data sets
y_train = np.array(np.split(pd.DataFrame(data[:,8]).to_numpy(), num)) # y_train as array of 100 training data sets

print(x_train.shape)
print(y_train.shape)

(10, 100000, 8)
(10, 100000, 1)


Now we fit 100 neural networks to its corresponding training data:

In [ ]:
NN_models = []
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=10**(-5), patience=5) # Early stopping criterion

for i in range(num):
    print('Iteration:',i + 1)
    NN_models.append(tf.keras.models.Sequential())
    NN_models[i].add(tf.keras.layers.Dense(neurons, activation=tf.nn.sigmoid, input_dim = d)) 
    NN_models[i].add(tf.keras.layers.Dense(1))

    NN_models[i].compile(optimizer='Adam', loss='mse')

    # Training process

    NN_models[i].fit(x_train[i], y_train[i], batch_size=32, epochs=150, validation_data=(x_val, y_val), callbacks=[callback], verbose=0) # Training process

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8


Compute gradients for the neural networks:

In [ ]:
gradients = np.zeros((num,n,d))

for i in range(num):
    x = tf.constant(x_train[i], dtype=tf.float32) # We regard the gradient over the corresponding training data set
    with tf.GradientTape(persistent=True) as tape: # Calculate gradient of the neural network
        tape.watch(x)
        y = NN_models[i](x)

    gradients[i] = tape.gradient(y, x).numpy()

    del tape # Drop the reference to the tape

Now we compute the neural network test statistic:

In [ ]:
NN_test_stats = 1/n * (gradients**2).sum(axis=1) # Estimator for functional of neural network

np.set_printoptions(precision=6, suppress=True) # Set precision of output

print('Empirical neural network test statistic with respect to each variable: \n',NN_test_stats.shape)

Computing the test results:

In [ ]:
test_bool_arrays = (neurons*NN_test_stats > quantiles).astype(int)

Now to evaluate the power and size of the test we approximate the test's expectation by using the empirical expectation over the 250 samples:

In [ ]:
power_size = test_bool_arrays.sum(axis=0)/num # Average the test results over the 100 samples to approximate the test's expected value
print(power_size)

### Computing the B Estimator With Sampled Neural Network Test Statistics

The normal procedure goes as follows: We add a noise variable X_{d+1} in a new column to our data set. The corresponding new samples should possess a similar distribution to our original data. The next step is to split our data into a few alternative data sets. We use them to fit multiple nerual networks to each data set and compute the test statistic for X_{d+1}. Using this method we obtain (a few) samples of the neural network test statistic. Further, even if we do not know the regression function f_0, it is clear the the new variable X_{d+1} satisfies the null hypothesis, i.e. it is irrelevant for the regression output. Thus the convergence results apply and we have approxiamte samples of the limiting distribution. These samples are scaled by B^2. Now we generate samples from the unscaled series representation. By taking the average of the NN-test statistic samples and dividing by the average of the unscaled series representation samples, we obtain B^2. Note that this estimator is very noisy because creating NN-test statistic samples requires splitting our limited supply of training data.

Since repeating the prior process for an additional dimension is such a heavy task, we use X_8 as our noise variable. It possesses the same cahracteristics as any other noise variable mentioned above. Note however, that typically we do not know which input variables satisfy the null hypotheses. This is just a demonstration example.

Another point to mention is that in order to increase the likelihood of covering the Sobolev norm of f_0 with B, we take the maximum value of the NN-test statistic samples instead of the average.

In [ ]:
scaled_NN_test_stats_max = (neurons**2)*np.max(NN_test_stats[:,7]) # Only X_8 suffices the null hypothesis

unscaled_Z_mean = np.sum(unscaled_Z[:,7])

B = np.sqrt(scaled_NN_test_stats_max/unscaled_Z_mean)

print('B estimator:',B)

# T-test for Linear Regression Model

Now we evaluate the standard t-test for a linear regression model. For that we fit multiple models and thus obatin multiple samples of the t-test, which we need to estimate its power and size. This time we use the statsmodel library which provides a t-test for each variable with our null hypothesis.

In [ ]:
import statsmodels.api as sm

pvalues = np.zeros((num,d)) # Placeholder for probabilities of t-test

for i in range(num): # Generate multiple samples of t-test
    X = sm.add_constant(x_train[i]) # Statsmodels library requires one extra variable for intercept of linear regression
    results = sm.OLS(y_train[i], X).fit() # Fitting the linear regression model to its corresponding data set
    pvalues[i] = results.pvalues[1:] # Copy probabilities of t-test, pvalue of intercept not needed

Next we check if the pvalues of the test statistic are below our significance level alpha = 0.05. This represents the result of the t-test.

In [ ]:
lin_test_bool_array = (pvalues <= 0.05).astype(int) # Check if probability is below our alpha
power_size = lin_test_bool_array.sum(axis=0)/num # average the t-test samples to obtain empirical expectation
print(power_size)

# Correlated Feature Variable

In this last simulation experiment we investigate the robustness of our NN-test if the feature variable possesses a dependece structure. More specifically, we use a Gaussian copula with the same uniform margins as before. We obtain this distribution by sampling vectors from a multivariate normal distribution and applying a normal distribution function to each vector coordinate. Then Sklar's theorem ensures the uniformity of the marginal distributions. Our objective is again to evaluate the test's performance by estimating its power and size. First we define the mean and covariance matrix of the multivariate Gaussian distribution.

In [ ]:
from scipy.stats import norm # TODO

mean_zero = np.zeros(d) # Mean vector for multivariate normal distribution

# Define covariance matrix for multivariate normal distribution

Sig = np.eye(d)
Sig[0,1],Sig[1,0] = 0.1, 0.1
Sig[4,5],Sig[5,4] = 0.5, 0.5
Sig[3,6],Sig[6,3] = 0.3, 0.3

print(Sig)

Now we generate our data accordingly:

In [ ]:
n = 100000 # number of multivariate normal samples

mn_samples = np.random.multivariate_normal(mean_zero, Sig, num*n + m) # n samples from multivariate normal distribution
X = 2*norm.cdf(mn_samples) - 1 # Apply standard normal df to obtain uniform marginal distributions
eps = np.random.normal(0,sigma,num*n + m) # Gaussian noise

Y = 8 + X[:,0]**2 + X[:,1]*X[:,2] + np.cos(X[:,3]) + np.exp(X[:,4]*X[:,5]) + 0.1*X[:,6] + eps # Output computation
Y = Y.reshape((-1,1)) # Transpose output vector

data = np.concatenate((X,Y), axis=1) # First 8 columns as input and last column as output
data_df = pd.DataFrame(data) # Create dataframe of numpy matrix
data_df.columns = ['X_1','X_2','X_3','X_4','X_5','X_6','X_7','X_8','Y']
data_df

We split the data set into multiple training sets and a validation set:

In [ ]:
# Training data sets

x_train = np.array(np.split(pd.DataFrame(data[:n,0:d]).to_numpy(), num)) # x_train as array of 100 training data sets
y_train = np.array(np.split(pd.DataFrame(data[:n,d]).to_numpy(), num)) # y_train as array of 100 training data sets

# Validation data set

x_val = pd.DataFrame(data[n:n + m,0:d]).to_numpy()
y_val = pd.DataFrame(data[n:n + m,d]).to_numpy()

In [ ]:
NN_models = []
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=10**(-5), patience=5) # Early stopping criterion

for i in range(num):
    print('Iteration:',i + 1)
    
    # Model construction
    
    NN_models.append(tf.keras.models.Sequential())
    NN_models[i].add(tf.keras.layers.Dense(neurons, activation=tf.nn.sigmoid, input_dim = d)) 
    NN_models[i].add(tf.keras.layers.Dense(1))

    NN_models[i].compile(optimizer='Adam', loss='mse')

    # Training process
    NN_models[i].fit(x_train[i], y_train[i], batch_size=32, epochs=150, validation_data=(x_val, y_val), callbacks=[callback], verbose=0) # Training process

We calculate the gradients for each network over its training data set:

In [ ]:
gradients = np.zeros((num,n,d))

for i in range(num):
    x = tf.constant(x_train[i], dtype=tf.float32) # We regard the gradient over the corresponding training data set
    with tf.GradientTape(persistent=True) as tape: # Calculate gradient of the neural network
        tape.watch(x)
        y = NN_models[i](x)

    gradients[i] = tape.gradient(y, x).numpy()

    del tape # Drop the reference to the tape

Now we compute the expected value of the squared partial derivatives for each network, which represents the test statistic.

In [ ]:
NN_test_stats = 1/n * (gradients**2).sum(axis=1) # Estimator for functional of neural network

np.set_printoptions(precision=6, suppress=True) # Set precision of output

print('Empirical neural network test statistic with respect to each variable: \n',NN_test_stats.shape)

Next we check if the scaled NN-test statistic exceeds its quantile. This also represents the outcome of the test.

In [ ]:
test_bool_arrays = (neurons*NN_test_stats > quantiles).astype(int)

And lastly we average the test results to approximate the test's expected value, which represents its power and size.

In [ ]:
power_size = test_bool_arrays.sum(axis=0)/num # Average the test results over the 100 samples to approximate the test's expected value
print(power_size)